In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as torch_data
from torchsummary import summary
from torchvision import models
from pycocotools.coco import COCO
import torchvision.transforms as transforms
import json
import skimage.io as io
import matplotlib.pyplot as plt

import sys, os
sys.path.append(os.path.abspath("../"))

%load_ext autoreload
%autoreload 2


import datasets.coco.dataloaders as dataloaders


## Data loading